# **Kaggle Project**
## **Irem Nesli Erez & Ezgi Siir Kibris**

### April 10, 2022 with the time extension

This notebook preprocesses the training and test sets using mainly the nltk package. [1] Because the data sizes are big and we try various methods for the given classification problem, in order to save time on preparing the data on each file, we preprocess it once and then, continue with that version of the data.

In [1]:
# This section of the code is added by default by Kaggle.

# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
# Import the necessary packages

import nltk #language processing
from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk import pos_tag

import re
import emoji

from sklearn.feature_extraction.text import CountVectorizer #for bag of words

import gensim
from gensim.models import Word2Vec

from sklearn import preprocessing
from sklearn.decomposition import PCA

import matplotlib.pyplot as plt
import matplotlib as mpl
import seaborn as sns

from scipy import interpolate
from scipy.spatial import ConvexHull

from sklearn.manifold import SpectralEmbedding

from sklearn.manifold import TSNE

from sklearn.linear_model import LogisticRegressionCV

path = '/Users/nesli/Desktop/SPRING2022/DSCC465/Kaggle_istanbul/'
os.chdir(path)

import random
random.seed(465) #seed

## Read the data

In [3]:
# Training data set
data_train = pd.read_csv('congressional_tweet_training_data.csv')
data_train

,favorite_count,full_text,hashtags,retweet_count,year,party_id
0,0,"b""RT @KUSINews: One of our longtime viewers wa...",KUSI,10,2017.0,R
1,258,"b""Today I'm urging the @CDCgov to immediately ...",Coronavirus,111,2020.0,R
2,0,"b'Tomorrow, #MO03 seniors graduate from Calvar...",MO03,2,2014.0,R
3,9,b'Congrats to #TeamUSA and Canton Native @JGre...,TeamUSA WorldJuniors,3,2017.0,R
4,3,b'Pleased to support @amergateways at their Ju...,ImmigrantHeritageMonth,3,2019.0,D
...,...,...,...,...,...,...
592798,3,"b'This time, it focused on careers in #publics...",publicservice publicsafety,0,2017.0,R
592799,5,"b'.#StormyDaniels, #MichaelWolfe, #JamesComey ...",StormyDaniels MichaelWolfe JamesComey,1,2018.0,R
592800,33,b'@NRDems The American people deserve the trut...,CultureOfCorruption,14,2020.0,D
592801,4,b'Only 2 weeks left to submit your #app to the...,app copolitics CAC16 HouseOfCode co06,3,2016.0,R


In [4]:
# Test data set
data_test = pd.read_csv('congressional_tweet_test_data.csv')
data_test #party is set to D by default

,Id,favorite_count,full_text,hashtags,retweet_count,year,party
0,0,70,b'#TaxReform improved the playing field for Am...,TaxReform,13,2018.0,D
1,1,27,"b'This #NativeWomensEqualPay Day, we recommit ...",NativeWomensEqualPay,11,NaN,D
2,2,49,"b""\xe2\x80\x9cI became convinced that our gene...",MeToo ShatteringTheSilence,24,2017.0,D
3,3,14,"b'During #NationalAdoptionMonth, we honor the ...",NationalAdoptionMonth,2,2019.0,D
4,4,13,b'Happy #AirborneDay to our @USArmy paratroope...,AirborneDay AirborneAllTheWay,7,2018.0,D
...,...,...,...,...,...,...,...
264995,264995,516,b'We need to #ExtendCHIP before a single child...,ExtendCHIP,223,2017.0,D
264996,264996,0,"b""Our #ObamaCare investigation continued today...",ObamaCare,3,2013.0,D
264997,264997,1,b'Congratulations to the new #MissTeenUSA Loga...,MissTeenUSA CT,4,2012.0,D
264998,264998,2174,b'Speaking of dishonesty. Nothing like being c...,mosen,1168,2018.0,D


In [5]:
# Check for number of rows that have NaN [2]
data_train.isnull().any(axis=1).sum()

18712

In [6]:
data_train.isnull().any(axis=1).sum()/len(data_train)

0.03156529234838555

In [7]:
# Check for number of rows that have NaN [2]
data_test.isnull().any(axis=1).sum()

8347

In [8]:
data_test.isnull().any(axis=1).sum()/len(data_test)

0.03149811320754717

Since it is only $3\%$, we can drop the rows with "NaN" values. 

In [9]:
data_train=data_train.dropna()

In [10]:
data_test=data_test.dropna()

In [11]:
#Fix the indexing after dropping some rows [3]
data_train.reset_index(drop=True, inplace=True)

In [12]:
#Fix the indexing after dropping some rows [3]
data_test.reset_index(drop=True, inplace=True)

In [13]:
data_train

,favorite_count,full_text,hashtags,retweet_count,year,party_id
0,0,"b""RT @KUSINews: One of our longtime viewers wa...",KUSI,10,2017.0,R
1,258,"b""Today I'm urging the @CDCgov to immediately ...",Coronavirus,111,2020.0,R
2,0,"b'Tomorrow, #MO03 seniors graduate from Calvar...",MO03,2,2014.0,R
3,9,b'Congrats to #TeamUSA and Canton Native @JGre...,TeamUSA WorldJuniors,3,2017.0,R
4,3,b'Pleased to support @amergateways at their Ju...,ImmigrantHeritageMonth,3,2019.0,D
...,...,...,...,...,...,...
574086,3,"b'This time, it focused on careers in #publics...",publicservice publicsafety,0,2017.0,R
574087,5,"b'.#StormyDaniels, #MichaelWolfe, #JamesComey ...",StormyDaniels MichaelWolfe JamesComey,1,2018.0,R
574088,33,b'@NRDems The American people deserve the trut...,CultureOfCorruption,14,2020.0,D
574089,4,b'Only 2 weeks left to submit your #app to the...,app copolitics CAC16 HouseOfCode co06,3,2016.0,R


In [14]:
data_test

,Id,favorite_count,full_text,hashtags,retweet_count,year,party
0,0,70,b'#TaxReform improved the playing field for Am...,TaxReform,13,2018.0,D
1,2,49,"b""\xe2\x80\x9cI became convinced that our gene...",MeToo ShatteringTheSilence,24,2017.0,D
2,3,14,"b'During #NationalAdoptionMonth, we honor the ...",NationalAdoptionMonth,2,2019.0,D
3,4,13,b'Happy #AirborneDay to our @USArmy paratroope...,AirborneDay AirborneAllTheWay,7,2018.0,D
4,5,73,"b""Proud to join 200+ colleagues in SCOTUS amic...",LGBT,13,2017.0,D
...,...,...,...,...,...,...,...
256648,264995,516,b'We need to #ExtendCHIP before a single child...,ExtendCHIP,223,2017.0,D
256649,264996,0,"b""Our #ObamaCare investigation continued today...",ObamaCare,3,2013.0,D
256650,264997,1,b'Congratulations to the new #MissTeenUSA Loga...,MissTeenUSA CT,4,2012.0,D
256651,264998,2174,b'Speaking of dishonesty. Nothing like being c...,mosen,1168,2018.0,D


## Kodu hizlandirmak icin

data_train[0:100]

data_train=data_train[0:100]

data_test=data_test[0:100]

## Remove within

In [15]:
# Make the classes numerical

data_train['party_class']=data_train['party_id'].apply(lambda x: 0 if x=='D' else 1)

/var/folders/dt/96l3vfgd54s0dlvms03s3z400000gn/T/ipykernel_4377/552341189.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_train['party_class']=data_train['party_id'].apply(lambda x: 0 if x=='D' else 1)


In [16]:
data_test['party_class']=data_test['party'].apply(lambda x: 0 if x=='D' else 1)

/var/folders/dt/96l3vfgd54s0dlvms03s3z400000gn/T/ipykernel_4377/1761763176.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_test['party_class']=data_test['party'].apply(lambda x: 0 if x=='D' else 1)


## Clean the full_text column of the data sets.

For the following parts of the text preprocessing, we follow from our homework 4 codes [4], [5].

In [17]:
# Tokenize
tokens_list=[word_tokenize(sent) for sent in data_train['full_text']]

In [18]:
# Pos Tag 
tagged_tokens =[pos_tag(sent) for sent in tokens_list]

In [19]:
def get_wordnet_pos(treebank_tag): #Following from [6]

    if treebank_tag.startswith('J'):
        return wordnet.ADJ
    elif treebank_tag.startswith('V'):
        return wordnet.VERB
    elif treebank_tag.startswith('N'):
        return wordnet.NOUN
    elif treebank_tag.startswith('R'):
        return wordnet.ADV
    else:
        return None # for easy if-statement 

In [20]:
lemmatizer = WordNetLemmatizer()

In [21]:
x=pd.DataFrame(data_train['full_text'])

In [22]:
x['tagged_tokens']=tagged_tokens

In [23]:
lemma_list = x['tagged_tokens'].transform(lambda value: ' '.join([lemmatizer.lemmatize(a[0],pos=get_wordnet_pos(a[1])) if get_wordnet_pos(a[1]) else a[0] for a in  value]))

In [24]:
x['lemma_list']=lemma_list

In [25]:
x.head()

,full_text,tagged_tokens,lemma_list
0,"b""RT @KUSINews: One of our longtime viewers wa...","[(b, NN), ('', ''), (RT, NNP), (@, NNP), (KUSI...",b '' RT @ KUSINews : One of our longtime viewe...
1,"b""Today I'm urging the @CDCgov to immediately ...","[(b, NN), ('', ''), (Today, NN), (I, PRP), ('m...",b '' Today I 'm urge the @ CDCgov to immediate...
2,"b'Tomorrow, #MO03 seniors graduate from Calvar...","[(b'Tomorrow, NN), (,, ,), (#, #), (MO03, NNP)...","b'Tomorrow , # MO03 senior graduate from Calva..."
3,b'Congrats to #TeamUSA and Canton Native @JGre...,"[(b'Congrats, NNS), (to, TO), (#, #), (TeamUSA...",b'Congrats to # TeamUSA and Canton Native @ JG...
4,b'Pleased to support @amergateways at their Ju...,"[(b'Pleased, VBN), (to, TO), (support, VB), (@...",b'Pleased to support @ amergateways at their J...


In [26]:
# Remove numbers, words, characters, punctuation, links and emojis 

def emoji_free_text(text): # From [7] 
    return emoji.get_emoji_regexp().sub(r'', text)

In [27]:
def clean_text(text: str) -> str:
    stop_words = set(stopwords.words('english'))
    tokens = word_tokenize(text)
    cleaned_tokens = []
    for token in tokens:
        lower_token = token.lower()
        len_check = len(lower_token) >= 2
        #start_check = not (lower_token.startswith('http') or lower_token.startswith('\\') or lower_token.startswith(':'))
        stop_word_check = lower_token not in stop_words
        if len_check and stop_word_check:
            lower_token=re.sub(r'\d+', '', lower_token) 
            lower_token=re.sub(r'\b\w{1}\b', '',  lower_token)  
            lower_token=re.sub(r'[^\w\s]', '', lower_token)                # Remove punctuation
            lower_token=re.sub(r'http\S+', '',lower_token)                 # Remove links
            cleaned_tokens.append(re.sub('[,.!?]|<br \/>\+|<br \/>', '', lower_token))
    return ' '.join(cleaned_tokens)

In [28]:
Text_clean = x['lemma_list'].map(clean_text)
Text_cleaner = Text_clean .map(emoji_free_text)
data_train['text_clean']=Text_cleaner

/var/folders/dt/96l3vfgd54s0dlvms03s3z400000gn/T/ipykernel_4377/3666141621.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_train['text_clean']=Text_cleaner


In [29]:
data_train.head()

,favorite_count,full_text,hashtags,retweet_count,year,party_id,party_class,text_clean
0,0,"b""RT @KUSINews: One of our longtime viewers wa...",KUSI,10,2017.0,R,1,rt kusinews one longtime viewer congressman d...
1,258,"b""Today I'm urging the @CDCgov to immediately ...",Coronavirus,111,2020.0,R,1,today urge cdcgov immediately launch phone ...
2,0,"b'Tomorrow, #MO03 seniors graduate from Calvar...",MO03,2,2014.0,R,1,tomorrow mo senior graduate calvary lutheran f...
3,9,b'Congrats to #TeamUSA and Canton Native @JGre...,TeamUSA WorldJuniors,3,2017.0,R,1,congrats teamusa canton native jgreenway win w...
4,3,b'Pleased to support @amergateways at their Ju...,ImmigrantHeritageMonth,3,2019.0,D,0,pleased support amergateways june fiesta honor...


## Now, repeat the cleaning for the test data set

In [30]:
# Tokenize
tokens_list=[word_tokenize(sent) for sent in data_test['full_text']]

In [31]:
# Pos Tag 
tagged_tokens =[pos_tag(sent) for sent in tokens_list]

In [32]:
def get_wordnet_pos(treebank_tag): #Following from [6]

    if treebank_tag.startswith('J'):
        return wordnet.ADJ
    elif treebank_tag.startswith('V'):
        return wordnet.VERB
    elif treebank_tag.startswith('N'):
        return wordnet.NOUN
    elif treebank_tag.startswith('R'):
        return wordnet.ADV
    else:
        return None # for easy if-statement 

In [33]:
lemmatizer = WordNetLemmatizer()

In [34]:
x=pd.DataFrame(data_test['full_text'])

In [35]:
x['tagged_tokens']=tagged_tokens

In [36]:
lemma_list = x['tagged_tokens'].transform(lambda value: ' '.join([lemmatizer.lemmatize(a[0],pos=get_wordnet_pos(a[1])) if get_wordnet_pos(a[1]) else a[0] for a in  value]))

In [37]:
x['lemma_list']=lemma_list

In [38]:
Text_clean = x['lemma_list'].map(clean_text)
Text_cleaner = Text_clean .map(emoji_free_text)
data_test['text_clean']=Text_cleaner

/var/folders/dt/96l3vfgd54s0dlvms03s3z400000gn/T/ipykernel_4377/1101272201.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_test['text_clean']=Text_cleaner


In [39]:
data_test.head()

,Id,favorite_count,full_text,hashtags,retweet_count,year,party,party_class,text_clean
0,0,70,b'#TaxReform improved the playing field for Am...,TaxReform,13,2018.0,D,0,taxreform improve playing field american worke...
1,2,49,"b""\xe2\x80\x9cI became convinced that our gene...",MeToo ShatteringTheSilence,24,2017.0,D,0,xexci become convinced generation silence mak...
2,3,14,"b'During #NationalAdoptionMonth, we honor the ...",NationalAdoptionMonth,2,2019.0,D,0,during nationaladoptionmonth honor adoptive pa...
3,4,13,b'Happy #AirborneDay to our @USArmy paratroope...,AirborneDay AirborneAllTheWay,7,2018.0,D,0,happy airborneday usarmy paratrooper veteran s...
4,5,73,"b""Proud to join 200+ colleagues in SCOTUS amic...",LGBT,13,2017.0,D,0,proud join colleague scotus amicus brief arg...


In [40]:
# To save some memory space
del x

## Vectorize the data

Now for vectorization, we can use different techniques as presented in the lecture notes. [8] Try two different methods:

1. Bag of words
2. Word2vec

We will not try TF-IDF because it is rather beneficial for long text data.

Considering the fact that the bag of words method includes each unique word in the text as a dimension, it is better to apply vectorization on the merged data set that contains both the test and the training sets.

In [41]:
data=pd.concat([data_train,data_test], ignore_index=True)

In [42]:
data

,favorite_count,full_text,hashtags,retweet_count,year,party_id,party_class,text_clean,Id,party
0,0,"b""RT @KUSINews: One of our longtime viewers wa...",KUSI,10,2017.0,R,1,rt kusinews one longtime viewer congressman d...,NaN,NaN
1,258,"b""Today I'm urging the @CDCgov to immediately ...",Coronavirus,111,2020.0,R,1,today urge cdcgov immediately launch phone ...,NaN,NaN
2,0,"b'Tomorrow, #MO03 seniors graduate from Calvar...",MO03,2,2014.0,R,1,tomorrow mo senior graduate calvary lutheran f...,NaN,NaN
3,9,b'Congrats to #TeamUSA and Canton Native @JGre...,TeamUSA WorldJuniors,3,2017.0,R,1,congrats teamusa canton native jgreenway win w...,NaN,NaN
4,3,b'Pleased to support @amergateways at their Ju...,ImmigrantHeritageMonth,3,2019.0,D,0,pleased support amergateways june fiesta honor...,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
830739,516,b'We need to #ExtendCHIP before a single child...,ExtendCHIP,223,2017.0,NaN,0,we need extendchip single child lose health co...,264995.0,D
830740,0,"b""Our #ObamaCare investigation continued today...",ObamaCare,3,2013.0,NaN,0,obamacare investigation continue today demand...,264996.0,D
830741,1,b'Congratulations to the new #MissTeenUSA Loga...,MissTeenUSA CT,4,2012.0,NaN,0,congratulations new missteenusa logan west sou...,264997.0,D
830742,2174,b'Speaking of dishonesty. Nothing like being c...,mosen,1168,2018.0,NaN,0,speaking dishonesty nothing like catch camera ...,264998.0,D


In [ ]:
# Bag of words
# Following from [9]
vectorizer = CountVectorizer()

text_bow = vectorizer.fit_transform(data['text_clean'])
a = np.zeros((830744, 963153), dtype='uint8')
a=text_bow.toarray()
text_bow_df = pd.DataFrame(a,columns=vectorizer.get_feature_names())

In [ ]:
# Bag of words for hashtags
# Following from [9]
vectorizer = CountVectorizer()

hash_bow = vectorizer.fit_transform(data['hashtags'])
hash_bow_df = pd.DataFrame(hash_bow.toarray(),columns=vectorizer.get_feature_names())

In [ ]:
len_data=len(data)

In [ ]:
# Delete unnecessary variables

del data
del text_bow
del hash_bow

## Apply dimensionality reduction to bow_vector and hash_vector to decrease the dimensions to 3 principal components

We apply dimensionality reduction to both test and training data sets simultaneously following from our codes for HW6. [10], [11].

## PCA

In [ ]:
#Following from [12] to use sklearn's PCA 

#Standardize before calling PCA
scaler=preprocessing.StandardScaler().fit(text_bow_df)
text_bow_df_scaled=scaler.transform(text_bow_df)

sklearn_pca = PCA(n_components=3)
pcs = sklearn_pca.fit_transform(text_bow_df_scaled)

In [ ]:
# Add pcs to the original data
data_train['bow_pcs']=pcs[0:len(data_train)].tolist()

In [ ]:
data_test['bow_pcs']=pcs[len(data_train):len_data].tolist()

In [ ]:
bow_pc1=np.zeros(len(data_train))
bow_pc2=np.zeros(len(data_train))
bow_pc3=np.zeros(len(data_train))

In [ ]:
# Add the principal component returned
for i in np.arange(len(data_train)):
    bow_pc1[i]=data_train['bow_pcs'][i][0]
    bow_pc2[i]=data_train['bow_pcs'][i][1]
    bow_pc3[i]=data_train['bow_pcs'][i][2]

In [ ]:
data_train['bow_pc1']=bow_pc1
data_train['bow_pc2']=bow_pc2
data_train['bow_pc3']=bow_pc3

In [ ]:
#Add these columns to the test data
bow_pc1=np.zeros(len(data_test))
bow_pc2=np.zeros(len(data_test))
bow_pc3=np.zeros(len(data_test))

In [ ]:
for i in np.arange(len(data_test)):
    bow_pc1[i]=data_test['bow_pcs'][i][0]
    bow_pc2[i]=data_test['bow_pcs'][i][1]
    bow_pc3[i]=data_test['bow_pcs'][i][2]

In [ ]:
data_test['bow_pc1']=bow_pc1
data_test['bow_pc2']=bow_pc2
data_test['bow_pc3']=bow_pc3

In [ ]:
# visualization_code.py
# Directly copy-pasted from the provided file for earlier homework [13]

mpl.rcParams['figure.dpi'] = 600
plt.rcParams['figure.figsize'] = (20.0, 10.0)
plt.rcParams['font.family'] = "serif"
plt.rcParams['font.size'] = 10

df = data_train
pal = sns.color_palette("Paired")[:len(set(data_train['party_class']))]
p1 = sns.scatterplot(x="bow_pc1", y='bow_pc2', hue='party_class', palette = pal, data=df, s=250, alpha=0.7, legend=False)

#For each point, we add a text inside the bubble
for line in range(0,df.shape[0]):
     p1.text(df.bow_pc1[line], df.bow_pc2[line], df.party_class[line], horizontalalignment='left', size='medium', color='black', weight='semibold')

plt.suptitle('Two-Dimensional Map (PCA)', fontsize=36)
plt.xlabel('Dimension 1', fontsize=24)
plt.ylabel('Dimension 2', fontsize=24)


for i in df.party_class.unique():
    # get the convex hull
    points = df[df.party_class == i][['bow_pc1', 'bow_pc2']].values
    hull = ConvexHull(points)
    x_hull = np.append(points[hull.vertices,0],
                       points[hull.vertices,0][0])
    y_hull = np.append(points[hull.vertices,1],
                       points[hull.vertices,1][0])
    
    # interpolate
    dist = np.sqrt((x_hull[:-1] - x_hull[1:])**2 + (y_hull[:-1] - y_hull[1:])**2)
    dist_along = np.concatenate(([0], dist.cumsum()))
    spline, u = interpolate.splprep([x_hull, y_hull], 
                                    u=dist_along, s=0)
    interp_d = np.linspace(dist_along[0], dist_along[-1], 50)
    interp_x, interp_y = interpolate.splev(interp_d, spline)
    # plot shape
    plt.fill(interp_x, interp_y, '--', c=pal[i], alpha=0.2)
    

plt.grid()
plt.show()

## Spectral-embedding

Again, following from HW6 [10], [11].

In [ ]:
#Following from sklearn's guides [14]

embedding = SpectralEmbedding(n_components=3)
pcs_embedded = embedding.fit_transform(text_bow_df_scaled) #Use the scaled version of bow_vec column for all of the data

In [ ]:
data_train['bow_spec']=pcs_embedded[0:len(data_train)].tolist()

In [ ]:
data_test['bow_spec']=pcs_embedded[len(data_train):len_data].tolist()

In [ ]:
bow_pc1=np.zeros(len(data_train))
bow_pc2=np.zeros(len(data_train))
bow_pc3=np.zeros(len(data_train))

In [ ]:
for i in np.arange(len(data_train)):
    bow_pc1[i]=data_train['bow_spec'][i][0]
    bow_pc2[i]=data_train['bow_spec'][i][1]
    bow_pc3[i]=data_train['bow_spec'][i][2]

In [ ]:
data_train['bow_spec1']=bow_pc1
data_train['bow_spec2']=bow_pc2
data_train['bow_spec3']=bow_pc3

In [ ]:
#Add these columns to the test data

bow_pc1=np.zeros(len(data_test))
bow_pc2=np.zeros(len(data_test))
bow_pc3=np.zeros(len(data_test))

In [ ]:
for i in np.arange(len(data_test)):
    bow_pc1[i]=data_test['bow_spec'][i][0]
    bow_pc2[i]=data_test['bow_spec'][i][1]
    bow_pc3[i]=data_test['bow_spec'][i][2]

In [ ]:
data_test['bow_spec1']=bow_pc1
data_test['bow_spec2']=bow_pc2
data_test['bow_spec3']=bow_pc3

In [ ]:
# visualization_code.py
# Directly copy-pasted from the provided file

mpl.rcParams['figure.dpi'] = 600
plt.rcParams['figure.figsize'] = (20.0, 10.0)
plt.rcParams['font.family'] = "serif"
plt.rcParams['font.size'] = 10

df = data_train
pal = sns.color_palette("Paired")[:len(set(data_train['party_class']))]
p1 = sns.scatterplot(x="bow_spec1", y='bow_spec2', hue='party_class', palette = pal, data=df, s=250, alpha=0.7, legend=False)

#For each point, we add a text inside the bubble
for line in range(0,df.shape[0]):
     p1.text(df.bow_spec1[line], df.bow_spec2[line], df.party_class[line], horizontalalignment='left', size='medium', color='black', weight='semibold')

plt.suptitle('Two-Dimensional Map (Spectral Embedding)', fontsize=36)
plt.xlabel('Dimension 1', fontsize=24)
plt.ylabel('Dimension 2', fontsize=24)


for i in df.party_class.unique():
    # get the convex hull
    points = df[df.party_class == i][['bow_spec1', 'bow_spec2']].values
    hull = ConvexHull(points)
    x_hull = np.append(points[hull.vertices,0],
                       points[hull.vertices,0][0])
    y_hull = np.append(points[hull.vertices,1],
                       points[hull.vertices,1][0])
    
    # interpolate
    dist = np.sqrt((x_hull[:-1] - x_hull[1:])**2 + (y_hull[:-1] - y_hull[1:])**2)
    dist_along = np.concatenate(([0], dist.cumsum()))
    spline, u = interpolate.splprep([x_hull, y_hull], 
                                    u=dist_along, s=0)
    interp_d = np.linspace(dist_along[0], dist_along[-1], 50)
    interp_x, interp_y = interpolate.splev(interp_d, spline)
    # plot shape
    plt.fill(interp_x, interp_y, '--', c=pal[i], alpha=0.2)
    

plt.grid()
plt.show()

## t-SNE

Again, following from HW6 [10], [11].

In [ ]:
pcs_tsne = TSNE(n_components=3).fit_transform(text_bow_df_scaled)

In [ ]:
data_train['bow_tsne']=pcs_tsne[0:len(data_train)].tolist()

In [ ]:
data_test['bow_tsne']=pcs_tsne[len(data_train):len_data].tolist()

In [ ]:
bow_pc1=np.zeros(len(data_train))
bow_pc2=np.zeros(len(data_train))
bow_pc3=np.zeros(len(data_train))

In [ ]:
for i in np.arange(len(data_train)):
    bow_pc1[i]=data_train['bow_tsne'][i][0]
    bow_pc2[i]=data_train['bow_tsne'][i][1]
    bow_pc3[i]=data_train['bow_tsne'][i][2]

In [ ]:
data_train['bow_tsne1']=bow_pc1
data_train['bow_tsne2']=bow_pc2
data_train['bow_tsne3']=bow_pc3

In [ ]:
#Add these columns to the test data

bow_pc1=np.zeros(len(data_test))
bow_pc2=np.zeros(len(data_test))
bow_pc3=np.zeros(len(data_test))

In [ ]:
for i in np.arange(len(data_test)):
    bow_pc1[i]=data_test['bow_tsne'][i][0]
    bow_pc2[i]=data_test['bow_tsne'][i][1]
    bow_pc3[i]=data_test['bow_tsne'][i][2]

In [ ]:
data_test['bow_tsne1']=bow_pc1
data_test['bow_tsne2']=bow_pc2
data_test['bow_tsne3']=bow_pc3

In [ ]:
# visualization_code.py
# Directly copy-pasted from the provided file

mpl.rcParams['figure.dpi'] = 600
plt.rcParams['figure.figsize'] = (20.0, 10.0)
plt.rcParams['font.family'] = "serif"
plt.rcParams['font.size'] = 10

df = data_train
pal = sns.color_palette("Paired")[:len(set(data_train['party_class']))]
p1 = sns.scatterplot(x="bow_tsne1", y='bow_tsne2', hue='party_class', palette = pal, data=df, s=250, alpha=0.7, legend=False)

#For each point, we add a text inside the bubble
for line in range(0,df.shape[0]):
     p1.text(df.bow_tsne1[line], df.bow_tsne2[line], df.party_class[line], horizontalalignment='left', size='medium', color='black', weight='semibold')

plt.suptitle('Two-Dimensional Map (t-SNE)', fontsize=36)
plt.xlabel('Dimension 1', fontsize=24)
plt.ylabel('Dimension 2', fontsize=24)


for i in df.party_class.unique():
    # get the convex hull
    points = df[df.party_class == i][['bow_tsne1', 'bow_tsne2']].values
    hull = ConvexHull(points)
    x_hull = np.append(points[hull.vertices,0],
                       points[hull.vertices,0][0])
    y_hull = np.append(points[hull.vertices,1],
                       points[hull.vertices,1][0])
    
    # interpolate
    dist = np.sqrt((x_hull[:-1] - x_hull[1:])**2 + (y_hull[:-1] - y_hull[1:])**2)
    dist_along = np.concatenate(([0], dist.cumsum()))
    spline, u = interpolate.splprep([x_hull, y_hull], 
                                    u=dist_along, s=0)
    interp_d = np.linspace(dist_along[0], dist_along[-1], 50)
    interp_x, interp_y = interpolate.splev(interp_d, spline)
    # plot shape
    plt.fill(interp_x, interp_y, '--', c=pal[i], alpha=0.2)
    

plt.grid()
plt.show()

## Repeat dimensionality reduction for hashtags vectors

Again, following from HW6 [10], [11].

## PCA

In [ ]:
#Following from [12] to use sklearn's PCA 

#Standardize before calling PCA
scaler=preprocessing.StandardScaler().fit(hash_bow_df)
hash_bow_df_scaled=scaler.transform(hash_bow_df)

sklearn_pca = PCA(n_components=3)
pcs = sklearn_pca.fit_transform(hash_bow_df_scaled)

In [ ]:
# Add pcs to the original data
data_train['hash_pcs']=pcs[0:len(data_train)].tolist()

In [ ]:
data_test['hash_pcs']=pcs[len(data_train):len_data].tolist()

In [ ]:
bow_pc1=np.zeros(len(data_train))
bow_pc2=np.zeros(len(data_train))
bow_pc3=np.zeros(len(data_train))

In [ ]:
# Add the principal component returned
for i in np.arange(len(data_train)):
    bow_pc1[i]=data_train['hash_pcs'][i][0]
    bow_pc2[i]=data_train['hash_pcs'][i][1]
    bow_pc3[i]=data_train['hash_pcs'][i][2]

In [ ]:
data_train['hash_pc1']=bow_pc1
data_train['hash_pc2']=bow_pc2
data_train['hash_pc3']=bow_pc3

In [ ]:
#Add these columns to the test data
bow_pc1=np.zeros(len(data_test))
bow_pc2=np.zeros(len(data_test))
bow_pc3=np.zeros(len(data_test))

In [ ]:
for i in np.arange(len(data_test)):
    bow_pc1[i]=data_test['hash_pcs'][i][0]
    bow_pc2[i]=data_test['hash_pcs'][i][1]
    bow_pc3[i]=data_test['hash_pcs'][i][2]

In [ ]:
data_test['hash_pc1']=bow_pc1
data_test['hash_pc2']=bow_pc2
data_test['hash_pc3']=bow_pc3

## Spectral-embedding

In [ ]:
#Following from sklearn's guides [14]

embedding = SpectralEmbedding(n_components=3)
pcs_embedded = embedding.fit_transform(hash_bow_df_scaled) #Use the scaled version of bow_vec column for all of the data

In [ ]:
data_train['hash_spec']=pcs_embedded[0:len(data_train)].tolist()

In [ ]:
data_test['hash_spec']=pcs_embedded[len(data_train):len_data].tolist()

In [ ]:
bow_pc1=np.zeros(len(data_train))
bow_pc2=np.zeros(len(data_train))
bow_pc3=np.zeros(len(data_train))

In [ ]:
for i in np.arange(len(data_train)):
    bow_pc1[i]=data_train['hash_spec'][i][0]
    bow_pc2[i]=data_train['hash_spec'][i][1]
    bow_pc3[i]=data_train['hash_spec'][i][2]

In [ ]:
data_train['hash_spec1']=bow_pc1
data_train['hash_spec2']=bow_pc2
data_train['hash_spec3']=bow_pc3

In [ ]:
#Add these columns to the test data

bow_pc1=np.zeros(len(data_test))
bow_pc2=np.zeros(len(data_test))
bow_pc3=np.zeros(len(data_test))

In [ ]:
for i in np.arange(len(data_test)):
    bow_pc1[i]=data_test['hash_spec'][i][0]
    bow_pc2[i]=data_test['hash_spec'][i][1]
    bow_pc3[i]=data_test['hash_spec'][i][2]

In [ ]:
data_test['hash_spec1']=bow_pc1
data_test['hash_spec2']=bow_pc2
data_test['hash_spec3']=bow_pc3

## t-SNE

In [ ]:
pcs_tsne = TSNE(n_components=3).fit_transform(hash_bow_df_scaled)

In [ ]:
data_train['hash_tsne']=pcs_tsne[0:len(data_train)].tolist()

In [ ]:
data_test['hash_tsne']=pcs_tsne[len(data_train):len_data].tolist()

In [ ]:
bow_pc1=np.zeros(len(data_train))
bow_pc2=np.zeros(len(data_train))
bow_pc3=np.zeros(len(data_train))

In [ ]:
for i in np.arange(len(data_train)):
    bow_pc1[i]=data_train['hash_tsne'][i][0]
    bow_pc2[i]=data_train['hash_tsne'][i][1]
    bow_pc3[i]=data_train['hash_tsne'][i][2]

In [ ]:
data_train['hash_tsne1']=bow_pc1
data_train['hash_tsne2']=bow_pc2
data_train['hash_tsne3']=bow_pc3

In [ ]:
#Add these columns to the test data

bow_pc1=np.zeros(len(data_test))
bow_pc2=np.zeros(len(data_test))
bow_pc3=np.zeros(len(data_test))

In [ ]:
for i in np.arange(len(data_test)):
    bow_pc1[i]=data_test['hash_tsne'][i][0]
    bow_pc2[i]=data_test['hash_tsne'][i][1]
    bow_pc3[i]=data_test['hash_tsne'][i][2]

In [ ]:
data_test['hash_tsne1']=bow_pc1
data_test['hash_tsne2']=bow_pc2
data_test['hash_tsne3']=bow_pc3

## See how the data sets look like

In [ ]:
data_train.head()

In [ ]:
data_test.head()

## Prepare X_train, y_train

In [ ]:
data_train.columns

In [ ]:
X_train=data_train.loc[:, data_train.columns != 'party_id']

In [ ]:
X_train=X_train.loc[:, X_train.columns != 'full_text']

In [ ]:
X_train=X_train.loc[:, X_train.columns != 'party_class']

In [ ]:
X_train=X_train.loc[:, X_train.columns != 'text_clean']

In [ ]:
X_train=X_train.loc[:, X_train.columns != 'hashtags']

In [ ]:
X_train=X_train.loc[:, X_train.columns != 'bow_vector']

In [ ]:
X_train=X_train.loc[:, X_train.columns != 'bow_spec']

In [ ]:
X_train=X_train.loc[:, X_train.columns != 'bow_tsne']

In [ ]:
X_train=X_train.loc[:, X_train.columns != 'bow_pcs']

In [ ]:
X_train=X_train.loc[:, X_train.columns != 'hash_pcs']

In [ ]:
X_train=X_train.loc[:, X_train.columns != 'hash_spec']

In [ ]:
X_train=X_train.loc[:, X_train.columns != 'hash_tsne']

In [ ]:
X_train.columns

In [ ]:
y_train=data_train['party_class']

In [ ]:
y_train= pd.DataFrame(y_train)

In [ ]:
y_train

In [ ]:
X_train.to_csv('X_train.csv')

In [ ]:
del data_train

In [ ]:
del X_train

In [ ]:
y_train.to_csv('y_train.csv')

In [ ]:
del y_train

## Prepare X_test

In [ ]:
X_test=data_test.loc[:, data_test.columns != 'party_id']

In [ ]:
del data_test

In [ ]:
X_test=X_test.loc[:, X_test.columns != 'party']

In [ ]:
X_test=X_test.loc[:, X_test.columns != 'full_text']

In [ ]:
X_test=X_test.loc[:, X_test.columns != 'party_class']

In [ ]:
X_test=X_test.loc[:, X_test.columns != 'text_clean']

In [ ]:
X_test=X_test.loc[:, X_test.columns != 'hashtags']

In [ ]:
X_test=X_test.loc[:, X_test.columns != 'bow_vector']

In [ ]:
X_test=X_test.loc[:, X_test.columns != 'bow_spec']

In [ ]:
X_test=X_test.loc[:, X_test.columns != 'bow_tsne']

In [ ]:
X_test=X_test.loc[:, X_test.columns != 'bow_pcs']

In [ ]:
X_test=X_test.loc[:, X_test.columns != 'hash_pcs']

In [ ]:
X_test=X_test.loc[:, X_test.columns != 'hash_spec']

In [ ]:
X_test=X_test.loc[:, X_test.columns != 'hash_tsne']

In [ ]:
X_test #Id column is not considered as a column!

In [ ]:
Id=X_test['Id']

In [ ]:
Id.to_csv('Id.csv')

In [ ]:
del X_test['Id']

In [ ]:
X_test.to_csv('X_test.csv')

We could have ignored the warnings the code following from [15].

## References

[1] Bird, Steven, Edward Loper and Ewan Klein (2009), Natural Language Processing with Python. O’Reilly Media Inc.

[2] https://stackoverflow.com/questions/28199524/best-way-to-count-the-number-of-rows-with-missing-values-in-a-pandas-dataframe

[3] https://stackoverflow.com/questions/40755680/how-to-reset-index-pandas-dataframe-after-dropna-pandas-dataframe

[4] HW4 of Erez

[5] HW4 of Kibris

[6] Wordnet lemmatization and POS tagging in Python. Stack Overflow. Retrieved February 20, 2022, from https://stackoverflow.com/questions/15586721/wordnet-lemmatization-and-pos-tagging-in-python 

[7] Removing emojis from a string in Python. Stack Overflow. Retrieved February 20, 2022, from https://stackoverflow.com/questions/33404752/removing-emojis-from-a-string-in-python 

[8] Caliskan, Cantay. DSCC 465: Introduction to Statistical Machine Learning, Spring 2022, University of Rochester, Rochester NY.

[9] https://www.analyticsvidhya.com/blog/2021/08/a-friendly-guide-to-nlp-bag-of-words-with-python-example/

[10] HW6 of Erez

[11] HW6 of Kibris

[12] Galarnyk, M. (2021, November 17). PCA using Python (scikit-learn). Medium. Retrieved April 1, 2022, from https://towardsdatascience.com/pca-using-python-scikit-learn-e653f8989e60 

[13] visualization_code.py

[14] Scikit-learn: Machine Learning in Python, Pedregosa et al., JMLR 12, pp. 2825-2830, 2011.

[15] https://stackoverflow.com/questions/53784971/how-to-disable-convergencewarning-using-sklearn